# Appending to an Icechunk Store with Virtual References
## Note: This example uses a pre-2.0 release of VirtualiZarr
This notebook demonstrates how to append to an icechunk store.

Please ensure the correct dependencies are installed before starting.

In [1]:
# !pip install 'virtualizarr['icechunk','hdf']' ipykernel

In [2]:
!pip show icechunk

Name: icechunk
Version: 0.1.2
Summary: Icechunk Python
Home-page: https://github.com/earth-mover/icechunk
Author: Earthmover PBC
Author-email: Earthmover <info@earthmover.io>
License: Apache-2.0
Location: /opt/homebrew/envs/virtualizarr-tests/lib/python3.12/site-packages
Requires: zarr
Required-by: 


In [3]:
import warnings

import fsspec
import icechunk
import xarray as xr

from virtualizarr import open_virtual_dataset

warnings.filterwarnings("ignore", category=UserWarning)

# Before you start

Identify the dataset you will be using and create a list of files to generate a virtual icechunk datastore with.

In [4]:
fs = fsspec.filesystem("s3", anon=True)

oisst_files = fs.glob(
    "s3://noaa-cdr-sea-surface-temp-optimum-interpolation-pds/data/v2.1/avhrr/202408/oisst-avhrr-v02r01.*.nc"
)

oisst_files = sorted(["s3://" + f for f in oisst_files])

## Create virtual datasets with VirtualiZarr's `open_virtual_dataset`

In [5]:
so = dict(anon=True, default_fill_cache=False, default_cache_type="none")

virtual_datasets = [
    open_virtual_dataset(url, indexes={}, reader_options={"storage_options": so})
    for url in oisst_files[0:2]
]

In [6]:
virtual_ds = xr.concat(
    virtual_datasets,
    dim="time",
    coords="minimal",
    compat="override",
    combine_attrs="override",
)

In [7]:
virtual_ds

<xarray.Dataset> Size: 66MB
Dimensions:  (time: 2, zlev: 1, lat: 720, lon: 1440)
Coordinates:
    time     (time) float32 8B ManifestArray<shape=(2,), dtype=float32, chunk...
    zlev     (zlev) float32 4B ManifestArray<shape=(1,), dtype=float32, chunk...
    lat      (lat) float32 3kB ManifestArray<shape=(720,), dtype=float32, chu...
    lon      (lon) float32 6kB ManifestArray<shape=(1440,), dtype=float32, ch...
Data variables:
    sst      (time, zlev, lat, lon) float64 17MB ManifestArray<shape=(2, 1, 7...
    anom     (time, zlev, lat, lon) float64 17MB ManifestArray<shape=(2, 1, 7...
    err      (time, zlev, lat, lon) float64 17MB ManifestArray<shape=(2, 1, 7...
    ice      (time, zlev, lat, lon) float64 17MB ManifestArray<shape=(2, 1, 7...
Attributes: (12/37)
    Conventions:                CF-1.6, ACDD-1.3
    title:                      NOAA/NCEI 1/4 Degree Daily Optimum Interpolat...
    references:                 Reynolds, et al.(2007) Daily High-Resolution-...
    source:                     ICOADS, NCEP_GTS, GSFC_ICE, NCEP_ICE, Pathfin...
    id:                         oisst-avhrr-v02r01.20240801.nc
    naming_authority:           gov.noaa.ncei
    ...                         ...
    time_coverage_start:        2024-08-01T00:00:00Z
    time_coverage_end:          2024-08-01T23:59:59Z
    metadata_link:              https://doi.org/10.25921/RE9P-PT57
    ncei_template_version:      NCEI_NetCDF_Grid_Template_v2.0
    comment:                    Data was converted from NetCDF-3 to NetCDF-4 ...
    sensor:                     Thermometer, AVHRR

In [8]:
# Clean up the store if running this notebook multiple times.
#!rm -rf ./noaa-cdr-icechunk/

## Initialize the Icechunk Store

In [9]:
storage = icechunk.local_filesystem_storage("./noaa-cdr-icechunk")

config = icechunk.RepositoryConfig.default()

config.set_virtual_chunk_container(
    icechunk.VirtualChunkContainer("s3", "s3://", icechunk.s3_store(region="us-east-1"))
)

credentials = icechunk.containers_credentials(
    s3=icechunk.s3_credentials(anonymous=True)
)

repo = icechunk.Repository.open_or_create(storage, config, credentials)

session = repo.writable_session("main")

## Write the virtual datasets to the icechunk store and commit

In [10]:
virtual_ds.virtualize.to_icechunk(session.store)

In [11]:
session.commit("first 2 days of 202408 data")

'M9QAVC2ZG8MS9BVNGB80'

## Check your work!

In [12]:
ds = xr.open_zarr(session.store, consolidated=False, zarr_format=3)
ds

<xarray.Dataset> Size: 66MB
Dimensions:  (lon: 1440, time: 2, zlev: 1, lat: 720)
Coordinates:
  * lon      (lon) float32 6kB 0.125 0.375 0.625 0.875 ... 359.4 359.6 359.9
  * time     (time) datetime64[ns] 16B 2024-08-01T12:00:00 2024-08-02T12:00:00
  * lat      (lat) float32 3kB -89.88 -89.62 -89.38 -89.12 ... 89.38 89.62 89.88
  * zlev     (zlev) float32 4B 0.0
Data variables:
    sst      (time, zlev, lat, lon) float64 17MB dask.array<chunksize=(1, 1, 720, 1440), meta=np.ndarray>
    ice      (time, zlev, lat, lon) float64 17MB dask.array<chunksize=(1, 1, 720, 1440), meta=np.ndarray>
    anom     (time, zlev, lat, lon) float64 17MB dask.array<chunksize=(1, 1, 720, 1440), meta=np.ndarray>
    err      (time, zlev, lat, lon) float64 17MB dask.array<chunksize=(1, 1, 720, 1440), meta=np.ndarray>
Attributes: (12/37)
    Conventions:                CF-1.6, ACDD-1.3
    cdm_data_type:              Grid
    comment:                    Data was converted from NetCDF-3 to NetCDF-4 ...
    creator_email:              oisst-help@noaa.gov
    creator_url:                https://www.ncei.noaa.gov/
    date_created:               2024-08-16T09:12:00Z
    ...                         ...
    source:                     ICOADS, NCEP_GTS, GSFC_ICE, NCEP_ICE, Pathfin...
    standard_name_vocabulary:   CF Standard Name Table (v40, 25 January 2017)
    summary:                    NOAAs 1/4-degree Daily Optimum Interpolation ...
    time_coverage_end:          2024-08-01T23:59:59Z
    time_coverage_start:        2024-08-01T00:00:00Z
    title:                      NOAA/NCEI 1/4 Degree Daily Optimum Interpolat...

# Append

That was all nothing new! Basically a repeat of what is in the [icechunk docs](https://icechunk.io/icechunk-python/virtual/). Here we follow the same steps to create a virtual dataset, but we add an `append_dim` argument to the `to_icechunk` function.

In [13]:
virtual_datasets_a = [
    open_virtual_dataset(
        url, indexes={}, reader_options={"storage_options": {"anon": True}}
    )
    for url in oisst_files[2:4]
]

In [14]:
virtual_ds_a = xr.concat(
    virtual_datasets_a,
    dim="time",
    coords="minimal",
    compat="override",
    combine_attrs="override",
)

In [15]:
append_session = repo.writable_session("main")

In [16]:
virtual_ds_a.virtualize.to_icechunk(append_session.store, append_dim="time")

In [17]:
append_session.commit("wrote 2 more days of data")

'3MEW3ECB74ZYANAZZHT0'

# Check that it worked!

In [18]:
read_session = repo.readonly_session(branch="main")

In [19]:
ds = xr.open_zarr(read_session.store, consolidated=False, zarr_format=3)
ds

<xarray.Dataset> Size: 133MB
Dimensions:  (zlev: 1, time: 4, lat: 720, lon: 1440)
Coordinates:
  * zlev     (zlev) float32 4B 0.0
  * lat      (lat) float32 3kB -89.88 -89.62 -89.38 -89.12 ... 89.38 89.62 89.88
  * time     (time) datetime64[ns] 32B 2024-08-01T12:00:00 ... 2024-08-04T12:...
  * lon      (lon) float32 6kB 0.125 0.375 0.625 0.875 ... 359.4 359.6 359.9
Data variables:
    ice      (time, zlev, lat, lon) float64 33MB dask.array<chunksize=(1, 1, 720, 1440), meta=np.ndarray>
    err      (time, zlev, lat, lon) float64 33MB dask.array<chunksize=(1, 1, 720, 1440), meta=np.ndarray>
    sst      (time, zlev, lat, lon) float64 33MB dask.array<chunksize=(1, 1, 720, 1440), meta=np.ndarray>
    anom     (time, zlev, lat, lon) float64 33MB dask.array<chunksize=(1, 1, 720, 1440), meta=np.ndarray>
Attributes: (12/37)
    Conventions:                CF-1.6, ACDD-1.3
    cdm_data_type:              Grid
    comment:                    Data was converted from NetCDF-3 to NetCDF-4 ...
    creator_email:              oisst-help@noaa.gov
    creator_url:                https://www.ncei.noaa.gov/
    date_created:               2024-08-18T09:12:00Z
    ...                         ...
    source:                     ICOADS, NCEP_GTS, GSFC_ICE, NCEP_ICE, Pathfin...
    standard_name_vocabulary:   CF Standard Name Table (v40, 25 January 2017)
    summary:                    NOAAs 1/4-degree Daily Optimum Interpolation ...
    time_coverage_end:          2024-08-03T23:59:59Z
    time_coverage_start:        2024-08-03T00:00:00Z
    title:                      NOAA/NCEI 1/4 Degree Daily Optimum Interpolat...